# **Restaurant Inspections New York City**

## **Imports**

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly
%matplotlib inline

#Sklearn preprocessing
from sklearn import preprocessing,set_config
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder,StandardScaler,LabelEncoder
#Scipy
from scipy import stats
from scipy.stats import norm

#Sklearn Models
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn import preprocessing, set_config
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer,make_column_selector,make_column_transformer
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA


# Warnings 
import warnings 
warnings.filterwarnings("ignore")

# **Load Data**

In [2]:
rd = pd.read_csv('https://data.cityofnewyork.us/api/views/43nn-pn8j/rows.csv?accessType=DOWNLOAD')

# **Data Exploration**


In [3]:
# Check to see if data loaded properly:
rd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242682 entries, 0 to 242681
Data columns (total 27 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   CAMIS                  242682 non-null  int64  
 1   DBA                    241853 non-null  object 
 2   BORO                   242682 non-null  object 
 3   BUILDING               242284 non-null  object 
 4   STREET                 242668 non-null  object 
 5   ZIPCODE                239855 non-null  float64
 6   PHONE                  242679 non-null  object 
 7   CUISINE DESCRIPTION    239995 non-null  object 
 8   INSPECTION DATE        242682 non-null  object 
 9   ACTION                 239995 non-null  object 
 10  VIOLATION CODE         238662 non-null  object 
 11  VIOLATION DESCRIPTION  238662 non-null  object 
 12  CRITICAL FLAG          242682 non-null  object 
 13  SCORE                  230796 non-null  float64
 14  GRADE                  116932 non-nu

## **Drop Columns**

In [4]:
# Dropping irrelevant of imcomplete columns
rd.drop(['Location Point1', 'PHONE', 'BBL', 'BIN', 'NTA', 'Census Tract', 'Community Board', 'RECORD DATE'], axis=1, inplace=True)

In [5]:
# Check to see if data loaded properly:
rd.head()

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,INSPECTION TYPE,Latitude,Longitude,Council District
0,50155072,NALS NEST,Queens,12517,101ST AVE,11419.0,NaN,01/01/1900,NaN,NaN,NaN,Not Applicable,NaN,NaN,NaN,NaN,40.691461,-73.820692,28.0
1,50103964,RAS PLANT BASED,Brooklyn,739,FRANKLIN AVENUE,11238.0,Ethiopian,09/27/2021,Violations were cited in the following area(s).,04L,Evidence of mice or live mice present in facil...,Critical,53.0,NaN,NaN,Pre-permit (Operational) / Initial Inspection,40.673500,-73.956934,35.0
2,50143154,CRUMBL,Manhattan,195,BLEECKER STREET,10012.0,Bakery Products/Desserts,04/25/2024,Violations were cited in the following area(s).,10A,Toilet facility not maintained or provided wit...,Not Critical,11.0,A,04/25/2024,Pre-permit (Operational) / Re-inspection,40.729290,-74.001440,2.0
3,50051258,TASTI-D-LITE,Manhattan,1310,1 AVENUE,10021.0,Frozen Desserts,11/03/2022,Violations were cited in the following area(s).,10G,Dishwashing and ware washing: Cleaning and sa...,Not Critical,12.0,A,11/03/2022,Cycle Inspection / Initial Inspection,40.766750,-73.956748,5.0
4,50105575,COME ON THAI CUISINE,Brooklyn,7803,3 AVENUE,11209.0,Thai,02/22/2023,Violations were cited in the following area(s).,04L,Evidence of mice or live mice in establishment...,Critical,24.0,B,02/22/2023,Cycle Inspection / Re-inspection,40.629165,-74.028716,47.0


In [6]:
rd.to_csv('rd', index=False)